<a href="https://colab.research.google.com/github/0LinTungYi/40947036S/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import *
import time
import sys
import tensorflow as tf

In [ ]:
def read_input( round, count, end ):
    file = str(round)+'round.txt'
    input = []
    sol = []
    test_in = []
    test_sol = []
    with open( file, 'r') as infile:
        data = infile.readlines()
        sys.stdout = sys.stderr
        infile.close()
    
    c = 0
    key = 0
    temp = []
    for row in data:
        nums = row.split(" ")
        # 換行
        if(  len(nums) == 1 ):
            if(key==0):
                input.append(temp)
            else:
                test_in.append(temp)
            temp = []
            if( c==count ):
                key = 1
            elif ( c==end ):
                break
            c+=1
            
        # 分數 
        elif( len(nums) == 3 ):
            if( int(nums[0])>250 ):
                print( 'read_input error: score>250', int(nums[0]), c )
            elif( int(nums[0])<0 ):
                print( 'read_input error: score<0', int(nums[0]), c )
            if( key==0 ):
                sol.append( float(nums[0]) )
            else:
                test_sol.append( float(nums[0]) )        
        elif( len(nums) == 8 ):
            for i in range(7):
                temp.append( float(nums[i]) )
                if( int(nums[i])> 8 or int(nums[i])<0 ):
                    print( 'read_input error: chess>8', int(nums[i]) , c)
        else: 
            print( 'read_input error', nums, len(nums) )
    
    # sol2 = []
    # sol2.append( sol )
    # test_sol2 = []
    # test_sol2.append( test_sol )

    # return input, sol2, test_in, test_sol2
    return input, sol, test_in, test_sol



In [ ]:
def count_datas( round ):
    file = str(round)+'round.txt'
    
    with open( file, 'r') as infile:
        data = infile.readlines()
        sys.stdout = sys.stderr
        infile.close()
    
    count=0
    for row in data:
        nums = row.split(" ")
        # 換行
        if(  len(nums) == 1 ):
            count+=1
    
    return count


In [ ]:
def training_s1( train_in, train_sol, test_in, test_sol, num):
    
    train_in = array(train_in)
    train_sol = array(train_sol).T
    test_in = array(test_in)
    test_sol = array(test_sol).T
    print( train_in )
    print( train_sol )
    
    # initialize nn_weights
    random.seed(1)
    nn_weights1 =  2 * random.random((49,1)) - 1   # -1 ~ 1
    for i in range(num):
        print("\n i= ", i, "nn_weight1=")
        print(nn_weights1)
        
        # Calculate the outputs for each training examples
        z = dot(train_in, nn_weights1)
        train_out = 1 / (1 + exp(-z)) 
        
        # Run the NN adjustments 
        nn_weights1=nn_weights1+dot(train_in.T, (train_sol-train_out)*    train_out*(1-train_out)   ) 
        # print( dot(train_in.T, (train_sol-train_out)*    train_out*(1-train_out)   ) )
        # print("\n i= ", i, "MSE=")
        if i%2==0:
            print( ((train_sol*250 - train_out*250) ** 2).mean() )
            
    # Print the result for our unknown test input 
    test_out = 1 / (1 + exp(-(dot(test_in, nn_weights1))))
    print('\nThe final prediction is ', test_out*250 )
    print( ((test_out*250 - test_sol*250) ** 2).mean() )
    # print(test_out*250)
    # print(test_sol*250)
   

In [ ]:
x_train, y_train, x_test, y_test = read_input( 97, 900, 960 )
x_train = array(x_train)/7
y_train = array(y_train).T/250
x_test = array(x_test)/7
y_test = array(y_test).T/250
# print( count_datas(97) )
print(len(x_train))
print(x_train[0].shape) 
print(len(y_train))
print(y_train[0].shape) 
print(x_train)
# print(y_train)

901
(49,)
901
()
[[0.57142857 0.42857143 0.14285714 ... 0.42857143 0.85714286 0.28571429]
 [0.85714286 0.57142857 1.         ... 0.85714286 0.71428571 0.71428571]
 [0.71428571 0.57142857 0.57142857 ... 0.28571429 0.57142857 0.71428571]
 ...
 [0.57142857 0.71428571 0.85714286 ... 0.28571429 1.         0.57142857]
 [0.71428571 0.42857143 0.42857143 ... 0.57142857 0.14285714 0.71428571]
 [0.42857143 0.85714286 0.14285714 ... 1.         0.85714286 0.42857143]]


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(49, 1)),
  tf.keras.layers.Dense(8042, activation='relu'),
  #　随机的拿掉网络中的部分神经元，从而减小对W权重的依赖，以达到减小过拟合的效果。
  tf.keras.layers.Dropout(0.2),
  # tf.keras.layers.Dense(10)
  # 不用，這代表最後 output 為 10 個 class （0~9）的機率
])

In [ ]:
# predictions = model(x_train[:1]).numpy()
# predictions

In [ ]:
# tf.nn.softmax(predictions).numpy()
# 经过Softmax处理后，数值转化为相对概率

In [ ]:
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# loss_fn(y_train[:1], predictions).numpy()

In [ ]:
model.compile(# optimizer='adam',# Adam為目前較常使用的Optimizer
      optimizer='SGD',# stochastic gradient decent
      # loss=loss_fn,
      loss = 'sparse_categorical_crossentropy',
      metrics=['accuracy']) 

In [ ]:
model.fit(x_train, y_train, epochs=13)

Epoch 1/13
29/29 [==============================] - 1s 19ms/step - loss: 2.0176 - accuracy: 0.0055
Epoch 2/13
29/29 [==============================] - 1s 19ms/step - loss: 1.9576 - accuracy: 0.0055
Epoch 3/13
29/29 [==============================] - 1s 18ms/step - loss: 1.8977 - accuracy: 0.0055
Epoch 4/13
29/29 [==============================] - 1s 18ms/step - loss: 1.9475 - accuracy: 0.0055
Epoch 5/13
29/29 [==============================] - 1s 17ms/step - loss: 1.9574 - accuracy: 0.0055
Epoch 6/13
29/29 [==============================] - 1s 18ms/step - loss: 1.8775 - accuracy: 0.0055
Epoch 7/13
29/29 [==============================] - 1s 18ms/step - loss: 1.9274 - accuracy: 0.0055
Epoch 8/13
29/29 [==============================] - 1s 17ms/step - loss: 1.7477 - accuracy: 0.0055
Epoch 9/13
29/29 [==============================] - 1s 26ms/step - loss: 2.0870 - accuracy: 0.0055
Epoch 10/13
29/29 [==============================] - 1s 32ms/step - loss: 1.6379 - accuracy: 0.0055
Epoch 11/

In [ ]:
model.evaluate(x_test, y_test )

2/2 [==============================] - 0s 21ms/step - loss: 0.0032 - accuracy: 0.0000e+00


[0.0031969500705599785, 0.0]